In [1]:
# Load kedro environment (not needed in .py)
from pathlib import Path
from kedro.framework.context import load_context

# Load a context to be able to work in the notebook
#current_dir = Path.cwd()
current_dir = Path("/u01/share/cesar/aa_engine_uy/notebooks/")
proj_path = current_dir.parent
context = load_context(proj_path)
catalog = context.catalog
credentials = context.config_loader.get("credentials*","credentials*/**")
parameters = context.config_loader.get("parameters*","parameters*/**")

from aa_engine_pkg.assets.utils import *
from aa_engine_pkg.assets.core.data.kedro.catalog_expansion.partitioned_sql import SQLPartitionedDataSet

In [2]:
date='20180605'

In [3]:
def create_cliente_activo(cliente_activo: SQLPartitionedDataSet,
                          date: str) -> pd.DataFrame:
    """Creates master table with features related to EoP state of customers for one period of data

    Parameters
    ----------
    cliente_activo:
        dataset defined in ´catalog.yml´ - list of active customers at EoP for the given period
    date:
        period to process
    
    Returns
    -------
        Mastertable with information of clientes at EoP
    """

    # Initialize logger
    log = initialize_logger()

    # Load active clientes for period
    log.info(f"Creating cliente_activo...")
    period_to_load = get_previous_month(date)
    df_clientes_activos = cliente_activo.filter_by(date=period_to_load)

    # Return
    return df_clientes_activos

In [4]:
cliente_activo=catalog.load("cliente_activo")

2020-12-30 12:02:01,910 - kedro.io.data_catalog - INFO - Loading data from `cliente_activo` (SQLPartitionedDataSet)...


In [5]:
cliente_activo_df= create_cliente_activo(cliente_activo,date)

2020-12-30 12:02:04,513 - aa_engine_pkg.assets.utils.utilities - INFO - Creating cliente_activo...
select distinct CUSTOMER_ID from stg_uy_customer_status where UPPER(STATUS) LIKE '%ACTIVO%' and DATE_EXP = 201805


/u01/miniconda3/envs/uypo37/lib/python3.7/site-packages/sqlalchemy/dialects/oracle/base.py:1381: SAWarning: Oracle version (19, 5, 0, 0, 0) is known to have a maximum identifier length of 128, rather than the historical default of 30. SQLAlchemy 1.4 will use 128 for this database; please set max_identifier_length=128 in create_engine() in order to test the application with this new length, or set to 30 in order to assure that 30 continues to be used.  In particular, pay close attention to the behavior of database migrations as dynamically generated names may change. See the section 'Max Identifier Lengths' in the SQLAlchemy Oracle dialect documentation for background.
  % ((self.server_version_info,))


In [7]:
campanas= catalog.load("campanas")

2020-12-30 12:04:42,712 - kedro.io.data_catalog - INFO - Loading data from `campanas` (SQLPartitionedDataSet)...


In [8]:

vars_to_dummy = ["OFERTA_COMBINADA"]

past_periods = [14, 21, 28, 84, 168, 252, 336]

In [9]:
# Read parameters
log = initialize_logger()
look_back_days = parameters["masters"]["campanas"]["look_back_days"]
start_date = (pd.to_datetime(date) - timedelta(days=look_back_days)).strftime("%Y%m%d")

# Calculate period to load for active clients
period_to_load = get_previous_month(date)
log.info(f"Loading campanas...")
df_campanas = campanas.filter_by(date=[start_date, date]).drop_duplicates()

df_clientes = cliente_activo_df

df_campanas = pd.merge(df_clientes,
                       df_campanas,
                       on=["CUSTOMER_ID"],
                       how="inner")


2020-12-30 12:04:44,366 - aa_engine_pkg.assets.utils.utilities - INFO - Loading campanas...
select * from stg_uy_campana where START_DATE >= to_date('20170704', 'yyyymmdd') and START_DATE < to_date('20180605', 'yyyymmdd')


In [10]:
log.info("Creating variables...")
# Calculate offer duration / days since start / until end
df_campanas['OFFER_DURATION'] = (pd.to_datetime(df_campanas.END_DATE,
                                                errors="coerce") - df_campanas.START_DATE) / np.timedelta64(1,
                                                                                                            "D")
df_campanas["DAYS_SINCE_START_OFFER"] = (pd.to_datetime(date) - df_campanas.START_DATE) / np.timedelta64(1, "D")
df_campanas["DAYS_TO_END_OFFER"] = (pd.to_datetime(df_campanas.END_DATE, errors="coerce") - pd.to_datetime(
    date)) / np.timedelta64(1, "D")

# Extract percentages of offers from their description
df_campanas["PORC_OFFER"] = df_campanas["DESCRIPTION"].str.extract(r'(\d+)%')[0].astype(float)

# Extract discount values of offers from their description
df_campanas["VALUE_OFFER"] = df_campanas["DESCRIPTION"].str.extract(r'\$(\d+)')[0].astype(float)


2020-12-30 12:04:48,549 - aa_engine_pkg.assets.utils.utilities - INFO - Creating variables...


In [11]:
df_campanas.head()

,CUSTOMER_ID,START_DATE,END_DATE,ID,DESCRIPTION,SHUSERNR,EXTERNAL_AGENT_ID,USUARIO,DATE_EXP,OFFER_DURATION,DAYS_SINCE_START_OFFER,DAYS_TO_END_OFFER,PORC_OFFER,VALUE_OFFER
0,146099,2018-04-14,2018-04-15,4046,4046 - Upgrade Montevideo $0 Conexion,8026,NaN,ESALES_UY,201804,1.0,52.0,-51.0,NaN,0.0
1,150709,2018-02-12,2018-03-12,2534,2534-RETPREM-IN-HOTPACK 1 MES GRATIS+2 MESES(2...,8810,NaN,CO_MVALLEJO,201802,28.0,113.0,-85.0,NaN,NaN
2,150709,2018-03-12,2018-05-12,2533,2533 - VENTA HOTPACK 2 Meses al 50%,12,NaN,IBSINFRA_UY,201803,61.0,85.0,-24.0,50.0,NaN
3,150709,2018-05-19,2018-07-19,2544,2544-VENTAS PREM-FOX+/FOX HD FREE x 2 meses+ 2...,8810,NaN,CO_MVALLEJO,201805,61.0,17.0,44.0,50.0,NaN
4,150709,2018-04-19,2018-04-20,4046,4046 - Upgrade Montevideo $0 Conexion,8026,NaN,ESALES_UY,201804,1.0,47.0,-46.0,NaN,0.0


In [12]:
# Make a mask with position of values
maskvalue = df_campanas["VALUE_OFFER"].notna()
maskporc = df_campanas["PORC_OFFER"].notna()

# Create a unique ranking scaled for Percentage and value
maskall = dict(zip(["VALUE_OFFER", "PORC_OFFER"],
                   (maskvalue, maskporc)))
for var, mask in maskall.items():
    df_campanas.loc[mask, "MONTO_OFFER_SCALED"] = scale_values(df=df_campanas.loc[mask, :],
                                                               vars_to_groupby=["DATE_EXP"],
                                                               var_to_scale=var,
                                                               by_ranking=False)

    # Extract length of offer from their description
tuple_len = df_campanas["DESCRIPTION"].str.extract(r'(?:(\d+)[ ]*M|(\d+)X)', re.IGNORECASE)[[0, 1]].fillna(
    0).astype(int)

2020-12-30 12:04:51,372 - numexpr.utils - INFO - Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2020-12-30 12:04:51,373 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [13]:
df_campanas["LENGTH_OFFER"] = tuple_len.sum(axis=1)

In [14]:
# creo la variable tipo_oferta
condlist = [df_campanas["DESCRIPTION"].str.upper().str.contains("DSCTO|RET|DESC", na=False),
            df_campanas["DESCRIPTION"].str.upper().str.contains("RENTA", na=False),
            df_campanas["DESCRIPTION"].str.upper().str.contains("UPGRADE", na=False),
            df_campanas["DESCRIPTION"].str.upper().str.contains("NUEVO", na=False),
            df_campanas["DESCRIPTION"].str.upper().str.contains("RECON")
            ]
choicelist = ["RETE",
              "RENT",
              "UPGR",
              "NUEV",
              "RECO"]
df_campanas["TIPO_OFERTA"] = np.select(condlist, choicelist, default="OTROS")

In [15]:
# creo variable tipo de producto de la oferta
condlist = [df_campanas["DESCRIPTION"].str.upper().str.contains(
    "PREMIUM|FOX|HBO|EXXXOTICO|HOTPACK|ADULT|CLAXON|HUSTLER", na=False),
            df_campanas["DESCRIPTION"].str.upper().str.contains("PLATA|ORO|BRONCE|SICO", na=False),
            df_campanas["DESCRIPTION"].str.upper().str.contains("BUNDLE", na=False),
            df_campanas["DESCRIPTION"].str.upper().str.contains("NET", na=False),
            ]
choicelist = ["PREMIUM",
              "BASICO",
              "BUNDLE",
              "NET"]
df_campanas["PRODUCTO_OFERTA"] = np.select(condlist, choicelist, default="OTROS")

df_campanas["OFERTA_COMBINADA"] = df_campanas["TIPO_OFERTA"] + "_" + df_campanas["PRODUCTO_OFERTA"]
df_campanas["FECHA"] = df_campanas["START_DATE"].dt.strftime("%Y%m%d")

In [16]:
df_campanas.OFERTA_COMBINADA.unique()

array(['UPGR_OTROS', 'RETE_PREMIUM', 'OTROS_PREMIUM', 'RETE_BASICO',
       'RETE_OTROS', 'OTROS_OTROS', 'RENT_OTROS', 'OTROS_BASICO'],
      dtype=object)

In [17]:
# creo las variables dummy
df_list = []
for var in vars_to_dummy:
    log.info(f'---- {var}')
    df_list.append(group_categorical_variables(df_campanas,
                                               vars_to_groupby=["CUSTOMER_ID", "FECHA"],
                                               var_to_count=var,
                                               other_category=None))

# Reduce list of list to create a data table
df = reduce(lambda left, right: pd.merge(left, right, on=["CUSTOMER_ID", "FECHA"], how="outer"), df_list)

ofer_cols = [c for c in df.columns if "OFERTA" in c]

df["N_OFERTAS"] = df[ofer_cols].sum(axis=1)

# Join with num vars and expand
df_expanded_offers = add_relative_calculate_past(df=df_campanas[["CUSTOMER_ID",
                                                                 "FECHA",
                                                                 "DAYS_SINCE_START_OFFER",
                                                                 "DAYS_TO_END_OFFER",
                                                                 "LENGTH_OFFER",
                                                                 "MONTO_OFFER_SCALED"]],
                                                 id_cols=["CUSTOMER_ID"],
                                                 periods=past_periods,
                                                 agg=[np.nanmean, np.nanmin, np.nanmax],
                                                 date_col="FECHA",
                                                 start_date=start_date,
                                                 end_date=date,
                                                 period_freq="D")

# Join with num vars and expand
df_expanded_products = add_relative_calculate_past(df=df,
                                                   id_cols=["CUSTOMER_ID"],
                                                   periods=past_periods,
                                                   agg=[np.nansum],
                                                   date_col="FECHA",
                                                   start_date=start_date,
                                                   end_date=date,
                                                   period_freq="D")

# Merge
df_final = df_expanded_products.merge(df_expanded_offers, on="CUSTOMER_ID", validate="1:1")

# Add date variables
df_final["DATE_EXP"] = period_to_load
df_final["DATE_CALC"] = date


2020-12-30 12:07:36,579 - aa_engine_pkg.assets.utils.utilities - INFO - ---- OFERTA_COMBINADA
2020-12-30 12:07:36,833 - aa_engine_pkg.assets.utils.utilities - INFO - Adding relative date between 20170704 and 20180605
2020-12-30 12:07:36,878 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 14
2020-12-30 12:07:36,913 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 21
2020-12-30 12:07:36,955 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 28
2020-12-30 12:07:37,004 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 84
2020-12-30 12:07:37,101 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 168
2020-12-30 12:07:37,238 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 252
2020-12-30 12:07:37,371 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 336
2020-12-30 12:07:37,869 - aa_engine_pkg.assets.utils.utilit

In [18]:
df_final.head()

,CUSTOMER_ID,N_OFERTAS_nansum_14,OFERTA_COMBINADA_RETE_BASICO_nansum_14,OFERTA_COMBINADA_OTROS_PREMIUM_nansum_14,OFERTA_COMBINADA_RETE_OTROS_nansum_14,OFERTA_COMBINADA_RENT_OTROS_nansum_14,OFERTA_COMBINADA_OTROS_OTROS_nansum_14,OFERTA_COMBINADA_OTROS_BASICO_nansum_14,OFERTA_COMBINADA_UPGR_OTROS_nansum_14,OFERTA_COMBINADA_RETE_PREMIUM_nansum_14,...,MONTO_OFFER_SCALED_nanmin_336,MONTO_OFFER_SCALED_nanmax_336,DAYS_TO_END_OFFER_nanmean_336,DAYS_TO_END_OFFER_nanmin_336,DAYS_TO_END_OFFER_nanmax_336,DAYS_SINCE_START_OFFER_nanmean_336,DAYS_SINCE_START_OFFER_nanmin_336,DAYS_SINCE_START_OFFER_nanmax_336,DATE_EXP,DATE_CALC
0,8176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,-58.000000,-58.0,-58.0,59.000000,59.0,59.0,201805,20180605
1,8686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064516,0.064516,241.000000,241.0,241.0,151.000000,151.0,151.0,201805,20180605
2,8855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.208333,56.666667,-35.0,179.0,36.666667,26.0,48.0,201805,20180605
3,8864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,123.000000,-59.0,305.0,60.000000,60.0,60.0,201805,20180605
4,8882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,-61.000000,-61.0,-61.0,62.000000,62.0,62.0,201805,20180605


In [ ]:
def create_master_campanas(campanas: SQLPartitionedDataSet,
                           cliente_activo: pd.DataFrame,
                           parameters: Union[Dict, None],
                           date: str
                           ) -> pd.DataFrame:
    """Creates master table with offer features for one period of data
    Parameters
    ----------
    campanas:
        dataset defined in ``catalog_raw.yml`` with raw data information related to client's offers
    cliente_activo:
        dataset defined in ``catalog_raw.yml`` with raw data information related to active clients at EoP
    date:
        period to process
    parameters:
        set of project parameters defined in ``parameters.yml``
    Returns
    -------
    pd.DataFrame
        Master table with offer features for one period
    """

    # Read parameters
    log = initialize_logger()

    write_to_parquet = parameters["write_to_parquet"]
    table_name = "campanas"
    overwrite = parameters["masters"][table_name]["overwrite"]

    # Check if table was already created
    files = get_mastertable_paths_by_period(parameters=parameters, period=date)
    match = [str(file) for file in files if table_name in file]

    if len(match) > 0 and overwrite is False:
        # If table is found, read parquet:
        log.info(f"Reading {match[0]} table")
        df_final = pd.read_parquet(match[0], engine="pyarrow")

    else:
        # Read parameters
        log = initialize_logger()
        look_back_days = parameters["masters"]["campanas"]["look_back_days"]
        start_date = (pd.to_datetime(date) - timedelta(days=look_back_days)).strftime("%Y%m%d")

        # Calculate period to load for active clients
        period_to_load = get_previous_month(date)
        log.info(f"Loading campanas...")
        df_campanas = campanas.filter_by(date=[start_date, date]).drop_duplicates()

        df_clientes = cliente_activo

        df_campanas = pd.merge(df_clientes,
                               df_campanas,
                               on=["CUSTOMER_ID"],
                               how="inner")

        log.info("Creating variables...")
        # Calculate offer duration / days since start / until end
        df_campanas['OFFER_DURATION'] = (pd.to_datetime(df_campanas.END_DATE,
                                                        errors="coerce") - df_campanas.START_DATE) / np.timedelta64(1,
                                                                                                                    "D")
        df_campanas["DAYS_SINCE_START_OFFER"] = (pd.to_datetime(date) - df_campanas.START_DATE) / np.timedelta64(1, "D")
        df_campanas["DAYS_TO_END_OFFER"] = (pd.to_datetime(df_campanas.END_DATE, errors="coerce") - pd.to_datetime(
            date)) / np.timedelta64(1, "D")

        # Extract percentages of offers from their description
        df_campanas["PORC_OFFER"] = df_campanas["DESCRIPTION"].str.extract(r'(\d+)%')[0].astype(float)

        # Extract discount values of offers from their description
        df_campanas["VALUE_OFFER"] = df_campanas["DESCRIPTION"].str.extract(r'\$(\d+)')[0].astype(float)

        # Make a mask with position of values
        maskvalue = df_campanas["VALUE_OFFER"].notna()
        maskporc = df_campanas["PORC_OFFER"].notna()

        # Create a unique ranking scaled for Percentage and value
        maskall = dict(zip(["VALUE_OFFER", "PORC_OFFER"],
                           (maskvalue, maskporc)))
        for var, mask in maskall.items():
            df_campanas.loc[mask, "MONTO_OFFER_SCALED"] = scale_values(df=df_campanas.loc[mask, :],
                                                                       vars_to_groupby=["DATE_EXP"],
                                                                       var_to_scale=var,
                                                                       by_ranking=False)

            # Extract length of offer from their description
        tuple_len = df_campanas["DESCRIPTION"].str.extract(r'(?:(\d+)[ ]*M|(\d+)X)', re.IGNORECASE)[[0, 1]].fillna(
            0).astype(int)
        df_campanas["LENGTH_OFFER"] = tuple_len.sum(axis=1)

        # creo la variable tipo_oferta
        condlist = [df_campanas["DESCRIPTION"].str.upper().str.contains("DSCTO|RET|DESC", na=False),
                    df_campanas["DESCRIPTION"].str.upper().str.contains("RENTA", na=False),
                    df_campanas["DESCRIPTION"].str.upper().str.contains("UPGRADE", na=False),
                    df_campanas["DESCRIPTION"].str.upper().str.contains("NUEVO", na=False),
                    df_campanas["DESCRIPTION"].str.upper().str.contains("RECON")
                    ]
        choicelist = ["RETE",
                      "RENT",
                      "UPGR",
                      "NUEV",
                      "RECO"]
        df_campanas["TIPO_OFERTA"] = np.select(condlist, choicelist, default="OTROS")

        # creo variable tipo de producto de la oferta
        condlist = [df_campanas["DESCRIPTION"].str.upper().str.contains(
            "PREMIUM|FOX|HBO|EXXXOTICO|HOTPACK|ADULT|CLAXON|HUSTLER", na=False),
                    df_campanas["DESCRIPTION"].str.upper().str.contains("PLATA|ORO|BRONCE|SICO", na=False),
                    df_campanas["DESCRIPTION"].str.upper().str.contains("BUNDLE", na=False),
                    df_campanas["DESCRIPTION"].str.upper().str.contains("NET", na=False),
                    ]
        choicelist = ["PREMIUM",
                      "BASICO",
                      "BUNDLE",
                      "NET"]
        df_campanas["PRODUCTO_OFERTA"] = np.select(condlist, choicelist, default="OTROS")

        df_campanas["OFERTA_COMBINADA"] = df_campanas["TIPO_OFERTA"] + "_" + df_campanas["PRODUCTO_OFERTA"]
        df_campanas["FECHA"] = df_campanas["START_DATE"].dt.strftime("%Y%m%d")

        # creo las variables dummy
        df_list = []
        for var in vars_to_dummy:
            log.info(f'---- {var}')
            df_list.append(group_categorical_variables(df_campanas,
                                                       vars_to_groupby=["CUSTOMER_ID", "FECHA"],
                                                       var_to_count=var,
                                                       other_category=None))

        # Reduce list of list to create a data table
        df = reduce(lambda left, right: pd.merge(left, right, on=["CUSTOMER_ID", "FECHA"], how="outer"), df_list)

        ofer_cols = [c for c in df.columns if "OFERTA" in c]

        df["N_OFERTAS"] = df[ofer_cols].sum(axis=1)

        # Join with num vars and expand
        df_expanded_offers = add_relative_calculate_past(df=df_campanas[["CUSTOMER_ID",
                                                                         "FECHA",
                                                                         "DAYS_SINCE_START_OFFER",
                                                                         "DAYS_TO_END_OFFER",
                                                                         "LENGTH_OFFER",
                                                                         "MONTO_OFFER_SCALED"]],
                                                         id_cols=["CUSTOMER_ID"],
                                                         periods=past_periods,
                                                         agg=[np.nanmean, np.nanmin, np.nanmax],
                                                         date_col="FECHA",
                                                         start_date=start_date,
                                                         end_date=date,
                                                         period_freq="D")

        # Join with num vars and expand
        df_expanded_products = add_relative_calculate_past(df=df,
                                                           id_cols=["CUSTOMER_ID"],
                                                           periods=past_periods,
                                                           agg=[np.nansum],
                                                           date_col="FECHA",
                                                           start_date=start_date,
                                                           end_date=date,
                                                           period_freq="D")

        # Merge
        df_final = df_expanded_products.merge(df_expanded_offers, on="CUSTOMER_ID", validate="1:1")

        # Add date variables
        df_final["DATE_EXP"] = period_to_load
        df_final["DATE_CALC"] = date

        # Rename table
        rename_table(df=df_final,
                     preffix=parameters["masters"]["campanas"]["table_preffix"],
                     ids_to_exclude=["CUSTOMER_ID", "DATE_EXP", "DATE_CALC"]
                     )

        # Return
        log.info(f"Exporting {df_final.shape[0]} rows and {df_final.shape[1]} columns")

        if write_to_parquet:
            file = f"{parameters['paths']['master_path']}master_{table_name}/master_{table_name}_{date}.parquet"
            df_final.to_parquet(file, engine="pyarrow")

    return df_final